# 0.1-Simple-SequenceToSequence-RNN

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [6]:
import numpy as np

samples = 6
timesteps = 3
features = 1

x_train = np.array([
    [1, 1, 2], 
    [1, 2, 3], 
    [2, 3, 5], 
    [3, 5, 8], 
    [5, 8, 13], 
    [8, 13, 21]
])

y_train = np.array([
    [3, 5, 8], 
    [5, 8, 13], 
    [8, 13, 21], 
    [13, 21, 34], 
    [21, 34, 55], 
    [34, 55, 89]
])
print(x_train.shape)
print(y_train.shape)

(6, 3)
(6, 3)


In [7]:
# reshape input into [samples, timesteps, features]
x_train = x_train.reshape((6, 3, 1))
print(x_train.shape)
print(x_train)

(6, 3, 1)
[[[ 1]
  [ 1]
  [ 2]]

 [[ 1]
  [ 2]
  [ 3]]

 [[ 2]
  [ 3]
  [ 5]]

 [[ 3]
  [ 5]
  [ 8]]

 [[ 5]
  [ 8]
  [13]]

 [[ 8]
  [13]
  [21]]]


In [8]:
# reshape input into [samples, timesteps, features]
y_train = y_train.reshape((6, 3, 1))
print(y_train.shape)
print(y_train)

(6, 3, 1)
[[[ 3]
  [ 5]
  [ 8]]

 [[ 5]
  [ 8]
  [13]]

 [[ 8]
  [13]
  [21]]

 [[13]
  [21]
  [34]]

 [[21]
  [34]
  [55]]

 [[34]
  [55]
  [89]]]


In [54]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

# Define model
model = Sequential()
n_in = 3

# returns a sequence of vectors of dimension 100
model.add(LSTM(units=100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))

# returns a sequence of vectors of dimension 100
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))

# Compile model
model.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')

# fit model
model.fit(x_train, y_train, epochs=1000, batch_size=6, verbose=0)

# demonstrate recreation
yhat = model.predict(x_train, verbose=0)

In [55]:
print(yhat)
print(yhat[:,:,0])
print(np.round(yhat[:,:,0]))

[[[ 3.2451406]
  [ 5.0855417]
  [ 8.001696 ]]

 [[ 4.718005 ]
  [ 7.908141 ]
  [12.992541 ]]

 [[ 8.073333 ]
  [13.033678 ]
  [20.994324 ]]

 [[12.98729  ]
  [21.00585  ]
  [33.998844 ]]

 [[21.002686 ]
  [33.992188 ]
  [54.998837 ]]

 [[34.00139  ]
  [55.00249  ]
  [88.992325 ]]]
[[ 3.2451406  5.0855417  8.001696 ]
 [ 4.718005   7.908141  12.992541 ]
 [ 8.073333  13.033678  20.994324 ]
 [12.98729   21.00585   33.998844 ]
 [21.002686  33.992188  54.998837 ]
 [34.00139   55.00249   88.992325 ]]
[[ 3.  5.  8.]
 [ 5.  8. 13.]
 [ 8. 13. 21.]
 [13. 21. 34.]
 [21. 34. 55.]
 [34. 55. 89.]]


In [56]:
print(model.metrics_names)
score = model.evaluate(x_train, y_train, verbose=0)

['loss', 'acc']


In [57]:
print(score)

[0.009017043747007847, 1.0]
